In [ ]:
path_response = r'C:\Users\endwy\Documents\Personal\NYC Dog Data Project\data\Urban_Park_Ranger_Animal_Condition_Response.csv'
path_runs = r'C:\Users\endwy\Documents\Personal\NYC Dog Data Project\data\DPR_DogRuns_001.xml'

### df_license cleaning

In [ ]:
path_license = r'C:\Users\endwy\Documents\Personal\NYC Dog Data Project\data\NYC_Dog_Licensing_Dataset.csv'

import pandas as pd
with open(path_license):
    df_license = pd.read_csv(path_license, encoding='latin1')
df_license = df_license.sample(n=10)
# df_license.isna().sum()
df_license.dropna(axis=0,how='any',inplace=True)

In [ ]:
license_cols_to_str = ['ZipCode','CommunityDistrict','CensusTract2010','CityCouncilDistrict',
                       'CongressionalDistrict','StateSenatorialDistrict']

for i in license_cols_to_str:
    df_license[i] = df_license[i].apply(lambda x: str(x).split('.')[0])

In [ ]:
from datetime import datetime as dt
df_license.rename(columns={'AnimalBirthMonth':'AnimalBirthDate'},inplace=True)

df_license['AnimalBirthDate'] = df_license['AnimalBirthDate'].apply(lambda x: dt.strptime(x,'%m/%d/%Y %I:%M:%S %p'))
df_license['LicenseIssuedDate'] = df_license['LicenseIssuedDate'].apply(lambda x: dt.strptime(x,'%m/%d/%Y'))
df_license['LicenseExpiredDate'] = df_license['LicenseExpiredDate'].apply(lambda x: dt.strptime(x,'%m/%d/%Y'))
df_license['AnimalAge']=0

from dateutil import relativedelta as rd
today = pd.to_datetime(dt.now().date())

for i in range(len(df_license)):
    birth = df_license['AnimalBirthDate'].iloc[i]
    diff = rd.relativedelta(today, birth)
    years_diff = round(((diff.years*365 + diff.months*30 + diff.days)/365),2)
    df_license['AnimalAge'].iloc[i] = float(years_diff)

import numpy as np
df_license['LifeStatus']=''
for i in range(len(df_license)):
    if df_license['AnimalAge'].iloc[i]>20:
        df_license['LifeStatus'].iloc[i] = 'Deceased'
    elif df_license['AnimalAge'].iloc[i]<0:
        df_license['LifeStatus'].iloc[i] = np.nan
    else:
        df_license['LifeStatus'].iloc[i] = 'Alive'
        
df_license.dropna(axis=0,how='any',inplace=True)

### df_bite cleaning

In [ ]:
path_bite = r'C:\Users\endwy\Documents\Personal\NYC Dog Data Project\data\DOHMH_Dog_Bite_Data.csv'

import pandas as pd
with open(path_bite):
    df_bite = pd.read_csv(path_bite, encoding='latin1')
df_bite = df_bite.sample(n=1000)
# df_bite.isna().sum()
df_bite.dropna(axis=0,how='any',inplace=True)

In [ ]:
from dateutil.parser import parse as dtparse
for i in range(len(df_bite)):
    strings = df_bite['DateOfBite'].iloc[i].split()
    month = strings[0][:3]
    day = strings[1]
    year = strings[2]
    new_date = year+' '+month+' '+day
    df_bite['DateOfBite'].iloc[i] = dtparse(new_date)

from datetime import datetime as dt
df_bite['DateOfBite'] = pd.to_datetime(df_bite['DateOfBite'].apply(lambda x: dt.strftime(x,'%Y-%m-%d')))

In [ ]:
def age_to_num(string):
    import numpy as np
    string = str(string)
    
    if len(string.strip())==2 and string.strip()[-1].lower()=='y':
        return float(string.strip()[:-1])
    
    elif len(string.strip())==2 and string.strip()[-1].lower()=='m':
        return float(string.strip()[:-1])/12
    
    elif string.strip().lower().split()[-1]=='mnths':
        return float(string.strip().split()[0])/12
    
    elif string.strip().lower().split()[-1]=='yrs':
        return float(string.strip().split()[0])
    
    elif string.strip().lower()[:-2]=='wk':
        return float(string.strip().split('wk')[0])/52
    
    elif isinstance(string.strip(),float):
        return float(string.strip())
    
    else:
        return np.nan

for i in range(len(df_bite)):
    sliced = df_bite['Age'].iloc[i]
    df_bite['Age'].iloc[i] = round(age_to_num(sliced),2)

df_bite['Age'] = df_bite['Age'].apply(lambda x: pd.to_numeric(x))
df_bite.dropna(axis=0,how='any',inplace=True)